In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score


%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
pd.options.mode.chained_assignment = None  # default='warn'?

# data_key DataFrame
data_key = pd.read_csv('key.csv')

# data_train DataFrame
data_train = pd.read_csv('train.csv')

# data_weather DataFrame
data_weather = pd.read_csv('weather.csv')

rain_text = ['FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG', 'BR', 'UP', 'FG+']
other_text = ['HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SA', 'SS', 'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', 'BC', 'BL', 'VC' ]

data_weather['codesum'].replace("+", "")
a = []
for i in range(len(data_weather['codesum'])):
    a.append(data_weather['codesum'].values[i].split(" "))
    for i_text in a[i]:
        if len(i_text) == 4:
            a[i].append(i_text[:2])
            a[i].append(i_text[2:])
            
data_weather["nothing"] = 1
data_weather["rain"] = 0
data_weather["other"] = 0
b = -1
for ls in a:
    b += 1
    for text in ls:
        if text in rain_text:
            data_weather.loc[b, 'rain'] = 1
            data_weather.loc[b, 'nothing'] = 0
        elif text in other_text:
            data_weather.loc[b,'other'] = 1
            data_weather.loc[b, 'nothing'] = 0      
# 모든 데이터 Merge
df = pd.merge(data_weather, data_key)

station_nbr = df['station_nbr']
df.drop('station_nbr', axis=1, inplace=True)
df['station_nbr'] = station_nbr

df = pd.merge(df, data_train)

# T 값 처리 하기. Remained Subject = > 'M' and '-'
df['snowfall'][df['snowfall'] == '  T'] = 0.05
df['preciptotal'][df['preciptotal'] == '  T'] = 0.005

# 주말과 주중 구분 작업 하기
df['date'] = pd.to_datetime(df['date'])
df['week7'] = df['date'].dt.dayofweek
df['weekend'] = 0
df.loc[df['week7'] == 5, 'weekend'] = 1
df.loc[df['week7'] == 6, 'weekend'] = 1

df1 = df[df['station_nbr'] == 1]; df11 = df[df['station_nbr'] == 11]
df2 = df[df['station_nbr'] == 2]; df12 = df[df['station_nbr'] == 12]
df3 = df[df['station_nbr'] == 3]; df13 = df[df['station_nbr'] == 13]
df4 = df[df['station_nbr'] == 4]; df14 = df[df['station_nbr'] == 14]
df5 = df[df['station_nbr'] == 5]; df15 = df[df['station_nbr'] == 15]
df6 = df[df['station_nbr'] == 6]; df16 = df[df['station_nbr'] == 16]
df7 = df[df['station_nbr'] == 7]; df17 = df[df['station_nbr'] == 17]
df8 = df[df['station_nbr'] == 8]; df18 = df[df['station_nbr'] == 18]
df9 = df[df['station_nbr'] == 9]; df19 = df[df['station_nbr'] == 19]
df10 = df[df['station_nbr'] == 10]; df20 = df[df['station_nbr'] == 20]

In [3]:
df10 = df10.apply(pd.to_numeric, errors = 'coerce')
df10.describe().iloc[:, 14:]
# 없는 Column = date, depart, codesum, snowfall, station_nbr

,stnpressure,sealevel,resultspeed,resultdir,avgspeed,nothing,rain,other,store_nbr,station_nbr,item_nbr,units,week7,weekend
count,299034.000000,298701.000000,299034.000000,299034.000000,299034.000000,299034.000000,299034.000000,299034.000000,299034.000000,299034.0,299034.000000,299034.000000,299034.000000,299034.000000
mean,30.026347,30.052219,6.642094,16.824053,8.319376,0.491091,0.501114,0.076837,20.000000,10.0,56.000000,0.725690,2.995546,0.285078
std,0.129913,0.130016,3.512245,10.215959,2.953517,0.499921,0.500000,0.266334,6.683324,0.0,32.041693,5.856935,2.003336,0.451452
min,29.540000,29.570000,0.300000,1.000000,2.300000,0.000000,0.000000,0.000000,11.000000,10.0,1.000000,0.000000,0.000000,0.000000
25%,29.940000,29.970000,4.100000,9.000000,6.400000,0.000000,0.000000,0.000000,11.000000,10.0,28.000000,0.000000,1.000000,0.000000
50%,30.030000,30.050000,6.200000,17.000000,7.800000,0.000000,1.000000,0.000000,22.000000,10.0,56.000000,0.000000,3.000000,0.000000
75%,30.110000,30.130000,8.700000,24.000000,9.800000,1.000000,1.000000,0.000000,27.000000,10.0,84.000000,0.000000,5.000000,1.000000
max,30.420000,30.460000,21.500000,36.000000,21.900000,1.000000,1.000000,1.000000,27.000000,10.0,111.000000,220.000000,6.000000,1.000000


In [4]:
df10['store_nbr'].unique()

array([11, 22, 27], dtype=int64)

In [5]:
df10_drop_columns = ['codesum', 'station_nbr', 'date', 'snowfall', 'depart' ]
df10 = df10.drop(columns = df10_drop_columns)

In [6]:
# np.nan를 포함하고 있는 변수(column)를 찾아서, 그 변수에 mean 값 대입해서 Frame의 모든 Value가 fill 되게 하기.
df10_columns = df10.columns

# Cateogry 값을 포함하는 변수는 np.nan에 mode값으로 대체하고, 나머지 실수 값을 포함한 변수는 np.nan에 mean값으로 대체
for i in df10_columns:
    if (i == 'resultdir'):
        df10[i].fillna(df10[i].mode()[0], inplace=True)
        print(df10[i].mode()[0])
    else:
        df10[i].fillna(df10[i].mean(), inplace=True)

# 이제 모든 변수가 숫자로 표기 되었기 때문에, 가능 함. 
# 상대 습도 추가 #
df10['relative_humility'] = 100*(np.exp((17.625*((df10['dewpoint']-32)/1.8))/(243.04+((df10['dewpoint']-32)/1.8)))/np.exp((17.625*((df10['tavg']-32)/1.8))/(243.04+((df10['tavg']-32)/1.8))))

# 체감온도 계산
df10["windchill"] = 35.74 + 0.6215*df10["tavg"] - 35.75*(df10["avgspeed"]**0.16) + 0.4275*df10["tavg"]*(df10["avgspeed"]**0.16)

df10 = df10[df10['units'] != 0]

1


In [7]:
# 'np.log1p(units) ~ tmax + tmin + tavg + depart + dewpoint + wetbulb + heat + cool + sunrise + sunset + codesume + \
# snowfall + preciptotal + stnpressure + sealevel + resultspeed + resultdir + avgspeed + nothing + rain + other + \
# store_nbr + station_nbr + item_nbr'

model_df10 = sm.OLS.from_formula('np.log1p(units) ~ tmax + tmin + tavg + dewpoint + heat + cool + preciptotal + \
resultspeed + sealevel + resultdir + avgspeed + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + relative_humility + windchill + 0', data = df10)
result_df10 = model_df10.fit()

print(result_df10.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.802
Model:                            OLS   Adj. R-squared:                  0.801
Method:                 Least Squares   F-statistic:                     1016.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        15:42:07   Log-Likelihood:                -6595.2
No. Observations:                8073   AIC:                         1.326e+04
Df Residuals:                    8040   BIC:                         1.349e+04
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(nothing)[0]         -0.0089      0

In [12]:
model_df10 = sm.OLS.from_formula('np.log1p(units) ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(heat) + scale(cool) + \
scale(preciptotal) + scale(resultspeed) + scale(sealevel) + scale(resultdir) + scale(avgspeed) + C(nothing) + C(rain) + \
C(other) + C(item_nbr) + C(week7) + C(weekend) + scale(relative_humility) + scale(windchill) + 0', data = df10)
result_df10 = model_df10.fit()

print(result_df10.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.802
Model:                            OLS   Adj. R-squared:                  0.801
Method:                 Least Squares   F-statistic:                     1016.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        15:46:41   Log-Likelihood:                -6595.2
No. Observations:                8073   AIC:                         1.326e+04
Df Residuals:                    8040   BIC:                         1.349e+04
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(nothing)[0]           

In [13]:
anova_result_df10 = sm.stats.anova_lm(result_df10).sort_values(by=['PR(>F)'], ascending = False)
anova_result_df10[anova_result_df10['PR(>F)'] <= 0.05]

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
scale(windchill),1.0,2.188652,2.188652,7.265817,7.042552e-03
C(other),1.0,2.193729,2.193729,7.282670,6.976844e-03
scale(resultdir),1.0,2.981977,2.981977,9.899471,1.659298e-03
scale(avgspeed),1.0,4.711109,4.711109,15.639790,7.728289e-05
scale(preciptotal),1.0,5.715295,5.715295,18.973454,1.341946e-05
scale(tmax),1.0,9.759905,9.759905,32.400620,1.298553e-08
C(week7),6.0,150.511024,25.085171,83.276945,1.525933e-101
C(item_nbr),11.0,9609.859806,873.623619,2900.227679,0.000000e+00
C(nothing),2.0,61858.266807,30929.133404,102677.545422,0.000000e+00


In [14]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df10.values, i) for i in range(df10.shape[1])]
vif["features"] = df10.columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)
vif
# 10순위까지 겹치는것만 쓴다
# C(other), resultdir, preciptotal, C(week7), C(item_nbr)

,VIF Factor,features
0,1.022080,store_nbr
1,1.163808,item_nbr
2,1.193088,units
3,1.271052,preciptotal
4,1.309192,other
5,1.391282,resultdir
6,2.800168,week7
7,2.838181,weekend
8,5.247752,sunset
9,6.231303,sunrise


In [15]:
# resultdir, preciptotal, C(rain), C(week7), C(item_nbr)
model_df10 = sm.OLS.from_formula('np.log1p(units) ~ C(item_nbr) + C(other) + C(week7)+ scale(resultdir) + scale(preciptotal) + 0', data = df10)
result_df10 = model_df10.fit()

print(result_df10.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.800
Model:                            OLS   Adj. R-squared:                  0.799
Method:                 Least Squares   F-statistic:                     1609.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        15:47:22   Log-Likelihood:                -6632.4
No. Observations:                8073   AIC:                         1.331e+04
Df Residuals:                    8052   BIC:                         1.345e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(item_nbr)[9]         3.5816      0

In [18]:
# resultdir, preciptotal, C(rain), C(week7), C(item_nbr)
model_df10 = sm.OLS.from_formula('np.log1p(units) ~ C(item_nbr) + C(other) + C(week7)+ scale(resultdir) + scale(preciptotal) + 0', data = df10)
result_df10 = model_df10.fit()

print(result_df10.summary())# resultdir, preciptotal, C(rain), C(week10), item_nbr, weekend
X10 = df10[['item_nbr', 'other', 'week7', 'preciptotal', 'resultdir']]
y10 = df10['units']
model10 = LinearRegression()

cv10 = KFold(n_splits=10, shuffle=True, random_state=0)

cross_val_score(model10, X10, y10, scoring="r2", cv=cv10)

PatsyError: Error evaluating factor: ValueError: cannot widen a non-numeric type dtype('O')
    np.log1p(units) ~ scale(snowfall) + scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(heat) + scale(cool) + scale(preciptotal) + scale(resultspeed) + scale(sealevel) + scale(resultdir) + scale(avgspeed) + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + C(weekend) + scale(relative_humility) + scale(windchill) + 0
                                        ^^^^^^^^^^^